In [2]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()

import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo

  Activating project at `c:\users\daniel\Documents\Python-Projects\ocrl-piano\Julia_fmincon_tests`
  No Changes to `C:\Users\Daniel\Documents\Python-Projects\ocrl-piano\Julia_fmincon_tests\Project.toml`
  No Changes to `C:\Users\Daniel\Documents\Python-Projects\ocrl-piano\Julia_fmincon_tests\Manifest.toml`


In [6]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [3]:
model = load_model("cartpole.xml")


MuJoCo Model with 5 bodies and 2 joints

In [4]:
function wrapped_mj_step(model, data, x, uk)    
    # given the current model and data. set the state and control to the model and perform a forward step
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[:] .= uk
        # set state
        data.qpos .= x[1:model.nq]
        data.qvel .= x[(model.nq + 1):end]
    else
        # if using diff types, we need to convert the dual numbers to floats
        converted_uk = ForwardDiff.value.(uk)
        converted_x = ForwardDiff.value.(x)
        # set control
        data.ctrl[:] .= converted_uk

        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end
    
    # take discrete dynamics step 
    step!(model, data) 

    # return updated state k + 1
    xkp1 = zeros(model.nq + model.nv) 
    xkp1 .= get_physics_state(model, data)
    
    return xkp1
end

function cartpole_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg = params.idx, params.N, params.xg
    Q, R, Qf = params.Q, params.R, params.Qf

    J = 0 
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        J += 0.5*(xi-xg)'*Q*(xi-xg)+0.5*ui'*R*ui
    end
    
    # dont forget terminal cost 
    xt = Z[idx.x[N]]
    J += 0.5*(xt-xg)'*Q*(xt-xg)
    return J 
end
function cartpole_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = init_data(model)
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # TODO: hermite simpson 
        # println("xi: ", xi[1:5])
        # print("ui: ", typeof(ui))
        xip1_mujoco = wrapped_mj_step(model, data, xi, ui)
        c[idx.c[i]] = xip1_mujoco - xip1
    end
    # println(typeof(c))
    return c 
end

function cartpole_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 

    con_1 = Z[idx.x[1]] - xic
    con_2 = Z[idx.x[N]] - xg

    return [con_1; con_2; cartpole_dynamics_constraints(params, Z)]
end
function cartpole_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    return zeros(eltype(Z), 0)
    # return inequality_constraints
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [9]:
model = load_model("cartpole.xml")
data = init_data(model)

# initiate time and time steps
dt = 0.01
tf = 1.0 
t_vec = 0:dt:tf 
N = length(t_vec)


Q = 1*diagm(ones(model.nq + model.nv))
R = 0.1*diagm(ones(model.nu))
Qf = 100*diagm(ones(model.nq + model.nv))

# indexing 
idx = create_idx(model.nq + model.nv, model.nu, N)

# initial and goal state
xic = [0, pi, 0.1, 0]
xg = [0, 0, 0, 0]

params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)


# primal bounds
x_l = -1 * Inf * ones(idx.nz)
x_u = Inf*ones(idx.nz)

x_l = -1*Inf*ones(idx.nz)
x_u = Inf*ones(idx.nz)
for i = 1:(N-1)
    x_l[idx.u[i]] .= -1.0
    x_u[idx.u[i]] .= 1.0
end
# inequality constraints
# c_l = -1 * Inf * ones(3*(idx.N-1))
# c_u = Inf * ones(3*(idx.N-1))
c_l = zeros(0)
c_u = zeros(0)
# initial guess 
z0 = 0.1*randn(idx.nz)

# choose diff type (try :auto, then use :finite if :auto doesn't work)
diff_type = :auto 
# diff_type = :finite


Z = fmincon(cartpole_cost,cartpole_equality_constraints,cartpole_inequality_constraints,
            x_l,x_u,c_l,c_u,z0,params, diff_type;
            tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose=true)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]


---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     2592
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian...

10-element Vector{Vector{Float64}}:
 [-0.00023834460521112303]
 [0.00015039653208903452]
 [1.376944046766262e-5]
 [0.00013323895078986604]
 [-5.107204601634793e-5]
 [-1.0710372227674348e-5]
 [0.00010944727210165364]
 [0.000672385265911858]
 [-0.0003412977411705185]
 [-0.00016866819294848867]